In order to run DeepSense on an embedded system with proper HW-SW Co-design, Deepsense is decomposed into 3 main sub-models:
The CNN, the RNN and the FC

In [2]:
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from functools import partial

import time
import pathlib
import math
import os
import sys
print(tf.__version__)

2.3.0-rc0


In [3]:
# Load the complete DeepSense Model
new_model = tf.keras.models.load_model('DeepSense')

# Show the model architecture
new_model.summary()


Model: "DeepSense"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
I1 (InputLayer)                 [(None, 20, 60, 1)]  0                                            
__________________________________________________________________________________________________
I2 (InputLayer)                 [(None, 20, 60, 1)]  0                                            
__________________________________________________________________________________________________
I1_conv1 (Conv2D)               (None, 20, 12, 64)   1088        I1[0][0]                         
__________________________________________________________________________________________________
I2_conv1 (Conv2D)               (None, 20, 12, 64)   1088        I2[0][0]                         
__________________________________________________________________________________________

In [8]:
# Recreate Inputs from the complete DeepSense Model
input1 = new_model.get_layer("I1")
input2 = new_model.get_layer("I2")

#Recreate a sub-model for CNN from the complete DeepSense Model
cnn1 = new_model.get_layer("I1_conv1")(input1.input)
cnn1 = new_model.get_layer("I1_BN1")(cnn1)
cnn1 = layers.ReLU(name="I1_ReLU1")(cnn1)
cnn1 = new_model.get_layer("I1_dropout1")(cnn1)
cnn1 = new_model.get_layer("I1_conv2")(cnn1)
cnn1 = new_model.get_layer("I1_BN2")(cnn1)
cnn1 = layers.ReLU(name="I1_ReLU2")(cnn1)
cnn1 = new_model.get_layer("I1_dropout2")(cnn1)
cnn1 = new_model.get_layer("I1_conv3")(cnn1)
cnn1 = new_model.get_layer("I1_BN3")(cnn1)
cnn1 = layers.ReLU(name="I1_ReLU3")(cnn1)
cnn1 = new_model.get_layer("I1_output")(cnn1)

cnn2 = new_model.get_layer("I2_conv1")(input2.input)
cnn2 = new_model.get_layer("I2_BN1")(cnn2)
cnn2 = layers.ReLU(name="I2_ReLU1")(cnn2)
cnn2 = new_model.get_layer("I2_dropout1")(cnn2)
cnn2 = new_model.get_layer("I2_conv2")(cnn2)
cnn2 = new_model.get_layer("I2_BN2")(cnn2)
cnn2 = layers.ReLU(name="I2_ReLU2")(cnn2)
cnn2 = new_model.get_layer("I2_dropout2")(cnn2)
cnn2 = new_model.get_layer("I2_conv3")(cnn2)
cnn2 = new_model.get_layer("I2_BN3")(cnn2)
cnn2 = layers.ReLU(name="I2_ReLU3")(cnn2)
cnn2 = new_model.get_layer("I2_output")(cnn2)

cnn = new_model.get_layer("concatenate")((cnn1,cnn2))

cnn = new_model.get_layer("Merge_dropout1")(cnn)
cnn = new_model.get_layer("reshape")(cnn)
cnn = new_model.get_layer("Merge_conv1")(cnn)
cnn = new_model.get_layer("Merge_BN1")(cnn)
cnn = layers.ReLU(name="Merge_ReLU1")(cnn)
cnn = new_model.get_layer("Merge_dropout2")(cnn)
cnn = new_model.get_layer("Merge_conv2")(cnn)
cnn = new_model.get_layer("Merge_BN2")(cnn)
cnn = layers.ReLU(name="Merge_ReLU2")(cnn)
cnn = new_model.get_layer("Merge_dropout3")(cnn)
cnn = new_model.get_layer("Merge_conv3")(cnn)
cnn = new_model.get_layer("Merge_BN3")(cnn)
cnn = layers.ReLU(name="Merge_ReLU3")(cnn)
cnn = new_model.get_layer("Merge_output")(cnn)
sub_cnn_model = keras.Model(inputs=(input1.input,input2.input), outputs=cnn, name="DeepSense_CNN")

#Recreate a sub-model for RNN from the complete DeepSense Model
rnn_input = keras.Input(shape=(cnn[0].shape),name="RNN_Input")
rnn = new_model.get_layer("GRU")(rnn_input)
rnn = new_model.get_layer("tf_op_layer_RealDiv")(rnn)
sub_rnn_model = keras.Model(inputs=rnn_input, outputs=rnn, name="DeepSense_RNN")

#Recreate a sub-model for FC from the complete DeepSense Model
fc_input = keras.Input(shape=(rnn[0].shape),name="FC_Input")
fc = new_model.get_layer("Output")(fc_input)
sub_fc_model = keras.Model(inputs=fc_input, outputs=fc, name="DeepSense_FC")

# Check all sub-models
sub_cnn_model.summary()
sub_rnn_model.summary()
sub_fc_model.summary()
keras.utils.plot_model(sub_cnn_model, "DeepSense_CNN.png", show_shapes=True)
keras.utils.plot_model(sub_rnn_model, "DeepSense_RNN.png", show_shapes=True)
keras.utils.plot_model(sub_fc_model, "DeepSense_FC.png", show_shapes=True)
sub_cnn_model.save("DeepSense_CNN.h5")
sub_rnn_model.save("DeepSense_RNN.h5")
sub_fc_model.save("DeepSense_FC.h5")

Model: "DeepSense_CNN"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
I1 (InputLayer)                 [(None, 20, 60, 1)]  0                                            
__________________________________________________________________________________________________
I2 (InputLayer)                 [(None, 20, 60, 1)]  0                                            
__________________________________________________________________________________________________
I1_conv1 (Conv2D)               (None, 20, 12, 64)   1088        I1[0][0]                         
__________________________________________________________________________________________________
I2_conv1 (Conv2D)               (None, 20, 12, 64)   1088        I2[0][0]                         
______________________________________________________________________________________

Load the DataSet to test the converted sub-models

In [5]:
SEPCTURAL_SAMPLES = 10
FEATURE_DIM = SEPCTURAL_SAMPLES*6*2
CONV_LEN = 3
CONV_LEN_INTE = 3#4
CONV_LEN_LAST = 3#5
CONV_NUM = 64
CONV_MERGE_LEN = 8
CONV_MERGE_LEN2 = 6
CONV_MERGE_LEN3 = 4
CONV_NUM2 = 64
INTER_DIM = 120
OUT_DIM = 6#len(idDict)
WIDE = 20
CONV_DROP_PROB = 0.2

BATCH_SIZE = 64
TOTAL_ITER_NUM = 1000000000

AUTOTUNE = tf.data.experimental.AUTOTUNE
VALID_FILENAMES = tf.io.gfile.glob("./DeepSenseData/eval.tfrecord")

def read_tfrecord(example):
    tfrecord_format = (
        {
            #"image": tf.io.FixedLenFeature([], tf.string),
            'label': tf.io.FixedLenFeature([OUT_DIM], tf.float32),
            'example': tf.io.FixedLenFeature([WIDE*FEATURE_DIM], tf.float32),
            #"target": tf.io.FixedLenFeature([], tf.int64),
        }
    )
    example = tf.io.parse_single_example(example, tfrecord_format)
    features = tf.expand_dims(example['example'], axis=0)
    features = tf.reshape(features, shape=(WIDE, FEATURE_DIM))
    features = tf.expand_dims(features, axis=2)
    print(features.shape)
    input1, input2 = tf.split(features, num_or_size_splits=2, axis=1)
    print(input1.shape)
    label = example['label']
    return (input1, input2) , label

def load_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(
        filenames
    )  # automatically interleaves reads from multiple files
    dataset = dataset.with_options(
        ignore_order
    )  # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(
        partial(read_tfrecord), num_parallel_calls=AUTOTUNE
    )
    # returns a dataset of (image, label) pairs if labeled=True or just images if labeled=False
    return dataset

def get_dataset(filenames):
    dataset = load_dataset(filenames)
    dataset = dataset.shuffle(1192)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

valid_dataset = get_dataset(VALID_FILENAMES)

for features, labels in valid_dataset.take(1):
    print(features[0].numpy()[0].shape)
    print(features[1].numpy()[0].shape)
    print(labels.numpy()[0])
    samples1=tf.expand_dims(features[0].numpy()[0], axis=0)
    samples2=tf.expand_dims(features[1].numpy()[0], axis=0)
    label=labels.numpy()[0]

(20, 120, 1)
(20, 60, 1)
(20, 60, 1)
(20, 60, 1)
[0. 0. 1. 0. 0. 0.]


Run a sample to check the accuracy of the sub-models chained together

In [9]:
cnn_output = sub_cnn_model.predict((samples1,samples2))
rnn_output = sub_rnn_model.predict(cnn_output)
output = sub_fc_model.predict(rnn_output)

print(np.argmax(output, axis=1))
print(np.argmax(label, axis=0))

[2]
2


Convert the sub-models into TFLite model files

In [10]:
cnn_converter = tf.lite.TFLiteConverter.from_keras_model(sub_cnn_model)
cnn_converter.optimizations = [tf.lite.Optimize.DEFAULT]

rnn_converter = tf.lite.TFLiteConverter.from_keras_model(sub_rnn_model)
rnn_converter.optimizations = [tf.lite.Optimize.DEFAULT]

fc_converter = tf.lite.TFLiteConverter.from_keras_model(sub_fc_model)
fc_converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = cnn_converter.convert()
tflite_model_file = pathlib.Path('DeepSense_cnn.tflite')
tflite_model_file.write_bytes(tflite_model)

tflite_model = rnn_converter.convert()
tflite_model_file = pathlib.Path('DeepSense_rnn.tflite')
tflite_model_file.write_bytes(tflite_model)

tflite_model = fc_converter.convert()
tflite_model_file = pathlib.Path('DeepSense_fc.tflite')
tflite_model_file.write_bytes(tflite_model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: C:\Users\mehdi\AppData\Local\Temp\tmpjz5rqxpu\assets
INFO:tensorflow:Assets written to: C:\Users\mehdi\AppData\Local\Temp\tmpqb28ftur\assets


INFO:tensorflow:Assets written to: C:\Users\mehdi\AppData\Local\Temp\tmpqb28ftur\assets


INFO:tensorflow:Assets written to: C:\Users\mehdi\AppData\Local\Temp\tmpalzpf4c_\assets


INFO:tensorflow:Assets written to: C:\Users\mehdi\AppData\Local\Temp\tmpalzpf4c_\assets


3904